In [1]:
import sqlite3
import pandas as pd
import numpy as np
import os
import requests

In [2]:
def requete_sql (requete) :
    try:
        connexion = sqlite3.connect('./data/liste_piezos.db')
        curseur = connexion.cursor()
        print("Connexion réussie à SQLite")
        curseur.execute(requete)
        connexion.commit()
        resultat = curseur.fetchall()
        curseur.close()
        connexion.close()
        print("Connexion SQLite est fermée")
        return resultat
    except sqlite3.Error as error:
        print("Erreur lors du mis à jour dans la table", error)

In [3]:
requete = "select code_bss from data_piezo"
liste_bss = [piezo[0] for piezo in requete_sql (requete)]

Connexion réussie à SQLite
Connexion SQLite est fermée


# Connexion à la bdd et création d'un dataframe

In [4]:
connexion = sqlite3.connect('./data/liste_piezos.db')
curseur = connexion.cursor()
requete = """
    SELECT * FROM data_piezo    
    WHERE date_debut < '2010-07-01'
    AND date_fin > '2010-07-01'
"""
df_piezo = pd.read_sql_query(requete, connexion, index_col='code_bss')
curseur.close()
connexion.close()

# Conversion des dates
df_piezo["date_debut"] = pd.to_datetime(df_piezo["date_debut"])
df_piezo["date_fin"] = pd.to_datetime(df_piezo["date_fin"])

In [5]:
df_piezo.head()

,date_debut,date_fin,nb_mesures_piezo,code_dpt,code_commune_insee,nom_commune,latitude,longitude,altitude_station,profondeur_investigation,codes_bdlisa,libelle_pe,codes_masse_eau_edl,noms_masse_eau_edl,fichier_piezo,fichier_era5,fichier_combined,commentaire
code_bss,,,,,,,,,,,,,,,,,,
03423X0056_100,1954-11-08,2022-05-30,3704,68,68113,Guémar,48.190195,7.396716,174.00,10.70,221AA15,P,CG101,"Nappe d’Alsace, Pliocène de Haguenau et Oligocène",0,0,0,None
06216X0228_P30-10,2006-10-19,2022-05-03,5129,03,03204,Paray-sous-Briailles,46.272865,3.401730,237.20,8.00,932AR01,P,GG128,Alluvions de l'Allier aval,0,0,0,None
05068X0028_SP010,1990-12-11,2022-05-11,11386,85,85083,Épine,46.990356,-2.240129,2.25,16.50,121AF01,L,GG036,Ile de Noirmoutier,0,0,0,None
09145X0057_N210,2002-01-09,2022-05-22,7248,84,84027,Caderousse,44.110203,4.751991,31.00,14.50,712IA01,C,DG352,Alluvions des plaines du Comtat (Aigues Lez),0,0,0,None
05483X0041_P210,2006-10-19,2011-03-17,1528,58,58225,Saincaize-Meauce,46.891482,3.067608,176.05,4.75,932AR01,p,GG128,Alluvions de l'Allier aval,0,0,0,None


# Fonction de collecte de la chronique d'un piezo

In [4]:
def f_data_piezo (code_bss:str, date_index:bool=True):
    """
    code_bss : code bss brut
    date_index : (default: True), date en index du dataframe
    return : csv file
    """
    # Nom de fichier
    file  = f'{code_bss}.csv'
    if not os.path.isfile(f'./data/piezo/{file}') : #si le fichier n'existe pas
        code_bss_transfo = code_bss.replace("_", "%2F", 1) #Nécessaire pour construire l'URL
        url = f"http://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss={code_bss_transfo}&size=20000"
        print(url)
        try :
            get = requests.get(url)
            json = get.json()
            data = json['data']
            #Construction du dataframe
            df = pd.DataFrame(columns=['date_mesure', 'piezo'])
            df['date_mesure'] = pd.concat([pd.DataFrame([mesure['date_mesure']], columns=['date_mesure']) for mesure in data], ignore_index=True)
            df["piezo"] = pd.concat([pd.DataFrame([float(mesure['profondeur_nappe'])*(-1)], columns=['piezo']) for mesure in data],ignore_index=True)
            df.set_index('date_mesure', inplace=True)
            df.to_csv(f'./data/piezo/{file}', sep=";", encoding='utf-8', )
            print(f"Fichier {file} enregistré dans data/piezo.")
        except Exception as e:
            print(f"{code_bss} : Problème durant la requête à l'API, {e}.")
            if e == "No objects to concatenate" :
                return False
            
    else : # Si le fichier existe déjà on le mentionne.
        print(f"Le fichier {file} existe déjà.")

In [5]:
url = f"http://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=03423X0056%2F100&size=20000"
get = requests.get(url)
json = get.json()
data = json['data']
data

[{'code_bss': '03423X0056/100',
  'urn_bss': 'http://services.ades.eaufrance.fr/pointeau/03423X0056/100',
  'date_mesure': '1954-11-08',
  'timestamp_mesure': -478051200000,
  'niveau_nappe_eau': 175.22,
  'mode_obtention': 'Valeur mesurée',
  'statut': 'Donnée contrôlée niveau 1',
  'qualification': 'Correcte',
  'code_continuite': '2',
  'nom_continuite': 'Point lié au point précédent',
  'code_producteur': '40494388800036',
  'nom_producteur': 'APRONA (40494388800036)',
  'code_nature_mesure': None,
  'nom_nature_mesure': None,
  'profondeur_nappe': 4.18},
 {'code_bss': '03423X0056/100',
  'urn_bss': 'http://services.ades.eaufrance.fr/pointeau/03423X0056/100',
  'date_mesure': '1954-11-15',
  'timestamp_mesure': -477446400000,
  'niveau_nappe_eau': 175.23,
  'mode_obtention': 'Valeur mesurée',
  'statut': 'Donnée contrôlée niveau 1',
  'qualification': 'Correcte',
  'code_continuite': '2',
  'nom_continuite': 'Point lié au point précédent',
  'code_producteur': '40494388800036',
  '

# Collecte de toutes les chroniques de la liste établie à partir de la bdd

In [8]:
# Création d'une black liste des piezos sans données accessibles
piezo_black_list = []

# Liste des code_bss de la bdd
requete = """
        SELECT code_bss FROM data_piezo 
        WHERE date_debut < '2010-07-01'
        AND date_fin > '2010-07-01'
"""
liste_bss = [piezo[0] for piezo in requete_sql(requete)]

for piezo in liste_bss :
    if f_data_piezo(piezo) == False:
        # Renvoi le nom du piezo qui n'a pas de données
        piezo_black_list.append(piezo)
    
# Suppresion des piezos de la black list de la base de données
requete_del_black_list = f"""
                        DELETE FROM data_piezo
                        WHERE code_bss IN {tuple(piezo_black_list)}
                        """
requete_sql(requete_del_black_list)
print("chargement terminé")

# Mise à jour de la base de donnéees (champ fichier_piezo)
bss_piezo = pd.Series([_.replace(".csv","") for _ in os.listdir(r"./data/piezo") if _.endswith(r".csv")], name="piezo")    
requete_fichier_piezo = f"""
        UPDATE data_piezo
        SET fichier_piezo =
        (CASE WHEN code_bss IN {tuple(bss_piezo)}
        THEN 1 ELSE 0 END)
        """
requete_sql(requete_fichier_piezo)
print("Base de données à jour")

Connexion réussie à SQLite
Connexion SQLite est fermée
Le fichier 03423X0056_100.csv existe déjà.
Le fichier 06216X0228_P30-10.csv existe déjà.
Le fichier 05068X0028_SP010.csv existe déjà.
Le fichier 09145X0057_N210.csv existe déjà.
Le fichier 05483X0041_P210.csv existe déjà.
Le fichier 07372X0048_F10.csv existe déjà.
Le fichier 06932X0178_P10.csv existe déjà.
Le fichier 09765X0157_R10.csv existe déjà.
Le fichier 06505X0093_PZ10.csv existe déjà.
Le fichier 05986X0103_P16-20.csv existe déjà.
Le fichier 07004X0046_D6-20.csv existe déjà.
Le fichier 02706X0074_S77-20.csv existe déjà.
Le fichier 08081X0026_SE.20.csv existe déjà.
Le fichier 04131X0111_120.csv existe déjà.
Le fichier 04137X0009_C20.csv existe déjà.
Le fichier 09654X0569_CNRP20.csv existe déjà.
Le fichier 04133X0006_30.csv existe déjà.
Le fichier 03786X0074_130.csv existe déjà.
Le fichier 09406X0643_P130.csv existe déjà.
Le fichier 02612X0021_S30.csv existe déjà.
Le fichier 04137X0022_S30.csv existe déjà.
Le fichier 03426X0223

http://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=01405X0212%2FFORAL1&size=20000
01405X0212_FORAL1 : Problème durant la requête à l'API, No objects to concatenate.
http://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=01405X0215%2FROSAL1&size=20000
01405X0215_ROSAL1 : Problème durant la requête à l'API, No objects to concatenate.
http://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=01398X0118%2FBETAL1&size=20000
01398X0118_BETAL1 : Problème durant la requête à l'API, No objects to concatenate.
Le fichier 01398X0099_GEL1.csv existe déjà.
Le fichier 07306X0066_PZEM1.csv existe déjà.
Le fichier 09513X0021_GMM1.csv existe déjà.
Le fichier 05013X0001_SM1.csv existe déjà.
Le fichier 05078X0040_PZN1.csv existe déjà.
Le fichier 09267X0014_PIEZO1.csv existe déjà.
Le fichier 00035X0210_PZO1.csv existe déjà.
Le fichier 06401X0010_P1.csv existe déjà.
Le fichier 00182X0010_P1.csv existe déjà.
Le fichier 08065X0010_P1.csv existe déjà.
Le fichier 0762

http://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=01653X0145%2FSTAVA2&size=20000
01653X0145_STAVA2 : Problème durant la requête à l'API, No objects to concatenate.
Le fichier 09652X0199_COURB2.csv existe déjà.
Le fichier 00225X0077_FC2.csv existe déjà.
Le fichier 02994X0138_SC2.csv existe déjà.
Le fichier 09508X0055_D2.csv existe déjà.
Le fichier 08614X0010_SCE2.csv existe déjà.
Le fichier 00461X0040_FE2.csv existe déjà.
Le fichier 02667X0014_FE2.csv existe déjà.
Le fichier 02883X0018_FE2.csv existe déjà.
Le fichier 01518X0139_FE2.csv existe déjà.
Le fichier 01195X0200_F2.csv existe déjà.
Le fichier 08503X0010_F2.csv existe déjà.
Le fichier 07332X0530_F2.csv existe déjà.
Le fichier 09768X0040_F2.csv existe déjà.
Le fichier 04552X0070_F2.csv existe déjà.
Le fichier 06533X0070_F2.csv existe déjà.
Le fichier 07733X0070_F2.csv existe déjà.
Le fichier 08032X0211_F2.csv existe déjà.
Le fichier 08774X0021_F2.csv existe déjà.
Le fichier 08268X0021_F2.csv existe déjà.
Le fich

Le fichier 08271X0299_PZ2.csv existe déjà.
Le fichier 06608X0027_AC-3.csv existe déjà.
Le fichier 09692X0128_PZ-3.csv existe déjà.
Le fichier 07097X0057_3.csv existe déjà.
Le fichier 07662X0278_45B-03.csv existe déjà.
Le fichier 10404X0056_S1003.csv existe déjà.
Le fichier 09665X0073_203.csv existe déjà.
Le fichier 03424X0029_203.csv existe déjà.
Le fichier 06988X0246_WL03.csv existe déjà.
Le fichier 01137X0172_PZ-13.csv existe déjà.
Le fichier 04454X0011_13.csv existe déjà.
Le fichier 00853X0030_PZ2013.csv existe déjà.
Le fichier 00471X0095_PZ2013.csv existe déjà.
Le fichier 01907X0068_PZ2013.csv existe déjà.
Le fichier 10402X0133_SRAE13.csv existe déjà.
Le fichier 08024X0070_F13.csv existe déjà.
Le fichier 04135X0140_M13.csv existe déjà.
Le fichier 07784X0079_PZEM13.csv existe déjà.
Le fichier 02594X0094_P13.csv existe déjà.
Le fichier 09758X0009_R13.csv existe déjà.
Le fichier 03577X0012_PZ13.csv existe déjà.
Le fichier 08014X0032_PZ13.csv existe déjà.
Le fichier 03081X0025_223.csv 

Le fichier 09402X0244_P185.csv existe déjà.
Le fichier 03422X0029_95.csv existe déjà.
Le fichier 00068X0010_F295.csv existe déjà.
Le fichier 10453X0295_P4795.csv existe déjà.
Le fichier 09663X0260_PA5.csv existe déjà.
Le fichier 08988X0014_D5.csv existe déjà.
Le fichier 10971X0155_PD5.csv existe déjà.
Le fichier 10035X0012_F5.csv existe déjà.
Le fichier 10396X0162_F5.csv existe déjà.
Le fichier 08035X0393_F5.csv existe déjà.
Le fichier 06162X0008_F5.csv existe déjà.
Le fichier 09387X0059_F5.csv existe déjà.
Le fichier 05342X0035_SF5.csv existe déjà.
Le fichier 02606X0112_G5.csv existe déjà.
Le fichier 07791X0204_PZEM5.csv existe déjà.
Le fichier 05078X0042_PZN5.csv existe déjà.
Le fichier 08296X0022_P5.csv existe déjà.
Le fichier 06932X0174_P5.csv existe déjà.
Le fichier 06301X0019_P5.csv existe déjà.
Le fichier 02215X0049_P5.csv existe déjà.
Le fichier 08465X0019_S5.csv existe déjà.
Le fichier 02682X0095_T5.csv existe déjà.
Le fichier 04854X0187_U5.csv existe déjà.
Le fichier 08905X10

Le fichier 07555X0071_FB.csv existe déjà.
Le fichier 07975X0081_PGB.csv existe déjà.
Le fichier 10795X0070_PZCOMB.csv existe déjà.
Le fichier 06288X0096_SB.csv existe déjà.
Le fichier 01372X0209_BRIEYB.csv existe déjà.
Le fichier 09382X0052_C.csv existe déjà.
Le fichier 06298X0035_F210C.csv existe déjà.
Le fichier 04132X0193_41C.csv existe déjà.
Le fichier 02725X0180_241C.csv existe déjà.
Le fichier 03428X0095_91C.csv existe déjà.
Le fichier 07208X0197_F1C.csv existe déjà.
Le fichier 03086X0024_212C.csv existe déjà.
Le fichier 03782X0113_65C.csv existe déjà.
Le fichier 01987X0009_337C.csv existe déjà.
Le fichier 03423X0017_209C.csv existe déjà.
Le fichier 02348X0009_319C.csv existe déjà.
Le fichier 04134X0095_39C.csv existe déjà.
Le fichier 00633X0132_FR.PAC.csv existe déjà.
Le fichier 09894X0077_DRAC.csv existe déjà.
Le fichier 07703X0043_SDC.csv existe déjà.
Le fichier 04713X0137_F.REC.csv existe déjà.
Le fichier 04103X0022_FC.csv existe déjà.
Le fichier 07415X0025_ESPINC.csv existe 

08592X0096_F : Problème durant la requête à l'API, No objects to concatenate.
Le fichier 09781X0007_F.csv existe déjà.
Le fichier 05612X0007_F.csv existe déjà.
Le fichier 00652X0007_F.csv existe déjà.
Le fichier 07823X0007_F.csv existe déjà.
Le fichier 02923X0007_F.csv existe déjà.
Le fichier 07833X0007_F.csv existe déjà.
Le fichier 07343X0007_F.csv existe déjà.
Le fichier 09773X0007_F.csv existe déjà.
Le fichier 04004X0007_F.csv existe déjà.
Le fichier 07704X0007_F.csv existe déjà.
Le fichier 06824X0007_F.csv existe déjà.
Le fichier 00465X0007_F.csv existe déjà.
Le fichier 08565X0007_F.csv existe déjà.
Le fichier 08915X1007_F.csv existe déjà.
Le fichier 10031X0017_F.csv existe déjà.
Le fichier 08281X0017_F.csv existe déjà.
Le fichier 06583X0017_F.csv existe déjà.
Le fichier 07346X0017_F.csv existe déjà.
Le fichier 03546X0017_F.csv existe déjà.
Le fichier 06837X0017_F.csv existe déjà.
Le fichier 08067X0017_F.csv existe déjà.
Le fichier 09267X0017_F.csv existe déjà.
Le fichier 07338X001

08904X1010_P : Problème durant la requête à l'API, No objects to concatenate.
Le fichier 10292X0110_P.csv existe déjà.
Le fichier 00584X0020_P.csv existe déjà.
Le fichier 00636X0020_P.csv existe déjà.
Le fichier 06376X0020_P.csv existe déjà.
Le fichier 03278X0020_P.csv existe déjà.
Le fichier 02921X0030_P.csv existe déjà.
Le fichier 00771X0030_P.csv existe déjà.
Le fichier 06377X0030_P.csv existe déjà.
Le fichier 07223X0130_P.csv existe déjà.
Le fichier 07807X0130_P.csv existe déjà.
Le fichier 06813X0040_P.csv existe déjà.
Le fichier 05687X0040_P.csv existe déjà.
Le fichier 07224X0140_P.csv existe déjà.
Le fichier 04738X0150_P.csv existe déjà.
Le fichier 02606X0060_P.csv existe déjà.
Le fichier 00176X0060_P.csv existe déjà.
Le fichier 05668X0080_P.csv existe déjà.
Le fichier 05441X0090_P.csv existe déjà.
Le fichier 01493X0001_P.csv existe déjà.
Le fichier 07184X0001_P.csv existe déjà.
Le fichier 05842X0011_P.csv existe déjà.
Le fichier 04333X0011_P.csv existe déjà.
Le fichier 05156X001

BSS003PAFM_X : Problème durant la requête à l'API, No objects to concatenate.
Le fichier BSS003MKHM_X.csv existe déjà.
Le fichier BSS003VEVM_X.csv existe déjà.
http://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=BSS003PAFQ%2FX&size=20000
BSS003PAFQ_X : Problème durant la requête à l'API, No objects to concatenate.
Le fichier BSS003UJHQ_X.csv existe déjà.
Le fichier BSS003VEUS_X.csv existe déjà.
Le fichier BSS004AXXS_X.csv existe déjà.
Le fichier BSS003ZKDU_X.csv existe déjà.
Le fichier BSS003UKJY_X.csv existe déjà.
Le fichier BSS003VETY_X.csv existe déjà.
Le fichier BSS003VFXY_X.csv existe déjà.
Le fichier 10882X0215_ADOUX.csv existe déjà.
Le fichier 09625X0231_LADOUX.csv existe déjà.
Le fichier 07231C0252_BUCLAY.csv existe déjà.
http://hubeau.eaufrance.fr/api/v1/niveaux_nappes/chroniques?code_bss=09126X0275%2FRABY&size=20000
09126X0275_RABY : Problème durant la requête à l'API, No objects to concatenate.
Le fichier 06537X0103_VEIGY.csv existe déjà.
Le fichier 07965X00

In [9]:
piezo_black_list

[]